## **Collegamento Google Drive**

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# **Valutazione Modelli**

## **Import librerie**

In [2]:
#utils
import numpy as np
import pandas as pd
import seaborn as sns
import joblib
import matplotlib.pyplot as plt

#modelli
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize

## Import dati

In [3]:
#import dati
data_combined = pd.read_csv('/content/drive/Shareddrives/biometria_progetto/combinated_data.csv')
keystotal = data_combined['Key']
data_combined2 = data_combined.drop(['Key'], axis=1)

#rimozione dei valori nulli
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
dati_imputed = pd.DataFrame(imputer.fit_transform(data_combined2), columns=data_combined2.columns)

#normalizzazione dei dati
normalised_data = pd.DataFrame(normalize(dati_imputed, axis = 0))
normalised_data['Key']= keystotal
dati_imputed['Key']= keystotal

# **MAIN** (Subject dependent)

In [4]:
# Caricamento delle etichette da un file CSV
etichette_train =pd.read_csv('/content/drive/Shareddrives/biometria_progetto/materiale_SubjectIndipendent/etichettetrain.csv')
etichette_test = pd.read_csv('/content/drive/Shareddrives/biometria_progetto/materiale_SubjectIndipendent/etichettetest.csv')

# Lista dei partecipanti
lista_partecipanti = ["ldy", "rx", "qyt", "tyc", "ha", "whh", "mz", "mhw", "zjy", "wll" ,"zjd", "hql", "ly", "wq", "cz"]

## **Funzione split randomico**

In [5]:
#siccome il set di dati è piccolo questa funzione mi garantisce che nella selezione randomica dei segnali ci siano 2 elementi di ogni classe per i dati di test
def divide_train_test(data, labels):
    classes = np.unique(labels)
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []

    for cls in classes:
        class_indices = np.where(labels == cls)[0]
        np.random.shuffle(class_indices)

        test_indices = class_indices[:2]
        train_indices = class_indices[2:]

        test_data.extend(data[test_indices])
        test_labels.extend(labels[test_indices])
        train_data.extend(data[train_indices])
        train_labels.extend(labels[train_indices])

    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

## **XGBOOST**

In [6]:
#Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_xgboost= 0.0
media_xgboost= 0.0
accuracyfinalxgboost= 0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=normalised_data[normalised_data['Key'].str.contains(f"{i}_")]
    #prendiamo un solo partecipante 
    data_par= data[data['Key'].str.contains(partecipante)]
    data_partecipant = data_par.drop(['Key'], axis=1)
    X_train, y_train, X_test, y_test = divide_train_test(np.array(data_partecipant), np.array(etichette_test))

    # Importa il modello migliore 
    best_xgboost_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_xgboost_model.pkl")
  
    # Addestramento dei con tutti i dati di addestramento
    best_xgboost_model.fit(X_train, np.ravel(y_train))

    xgboost_pred = best_xgboost_model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy_xgboost = accuracy_score(y_test, xgboost_pred)
  
    # Aggiornamento dell'accuratezza totale
    acc_xgboost += accuracy_xgboost
      
  # Calcolo della media per sessione
  media_xgboost = acc_xgboost / 15

  # Calcolo della media tra le 3 sessioni
  accuracyfinalxgboost += media_xgboost

# Stampa della media dell'accuratezza
print("Media dell'accuratezza XGBOOST:", accuracyfinalxgboost/3)

Media dell'accuratezza XGBOOST: 0.7805555555555556


## **SVM**

In [7]:
#Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_svm= 0.0
media_svm= 0.0
accuracyfinalvsm= 0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=normalised_data[normalised_data['Key'].str.contains(f"{i}_")]
    #prendiamo un solo partecipante 
    data_par= data[data['Key'].str.contains(partecipante)]
    data_partecipant = data_par.drop(['Key'], axis=1)
    X_train, y_train, X_test, y_test = divide_train_test(np.array(data_partecipant), np.array(etichette_test))

    # Importa il modello migliore 
    best_svm_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_svm_model.pkl")

    # Addestramento dei con tutti i dati di addestramento
    best_svm_model.fit(X_train,np.ravel(y_train))

    svm_pred= best_svm_model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy_svm = accuracy_score(y_test, svm_pred)
  
    # Aggiornamento dell'accuratezza totale
    acc_svm += accuracy_svm
      
  # Calcolo della media per sessione
  media_svm = acc_svm / 15

  # Calcolo della media tra le 3 sessioni
  accuracyfinalvsm += media_svm

# Stampa della media dell'accuratezza
print("Media dell'accuratezza SMV:", accuracyfinalvsm/3)

Media dell'accuratezza SMV: 0.6166666666666666


## **Random Forest**

In [8]:
# Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_rf= 0.0
media_rf= 0.0
accuracyfinalrf= 0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=normalised_data[normalised_data['Key'].str.contains(f"{i}_")]
    #prendiamo un solo partecipante 
    data_par= data[data['Key'].str.contains(partecipante)]
    data_partecipant = data_par.drop(['Key'], axis=1)
    X_train, y_train, X_test, y_test = divide_train_test(np.array(data_partecipant), np.array(etichette_test))

    # Importa il modello migliore 
    best_rf_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_rf_model.pkl")

    # Addestramento dei con tutti i dati di addestramento
    best_rf_model.fit(X_train, np.ravel(y_train))

    rf_pred=best_rf_model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy_rf = accuracy_score(y_test, rf_pred)

    # Aggiornamento dell'accuratezza totale
    acc_rf += accuracy_rf
      
  # Calcolo della media per sessione
  media_rf = acc_rf / 15

  # Calcolo della media tra le 3 sessioni
  accuracyfinalrf +=media_rf

# Stampa della media dell'accuratezza
print("Media dell'accuratezza Random Forest:", accuracyfinalrf/3)

Media dell'accuratezza Random Forest: 0.5527777777777778


## **Logistic Regression**

In [9]:
# Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_lr= 0.0
media_lr= 0.0
accuracyfinallr= 0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=dati_imputed[dati_imputed['Key'].str.contains(f"{i}_")]
    #prendiamo un solo partecipante 
    data_par= data[data['Key'].str.contains(partecipante)]
    data_partecipant = data_par.drop(['Key'], axis=1)
    X_train, y_train, X_test, y_test = divide_train_test(np.array(data_partecipant), np.array(etichette_test))

    # Importa il modello migliore 
    best_lr_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_lr_model.pkl")

    # Addestramento dei con tutti i dati di addestramento
    best_lr_model.fit(X_train,np.ravel(y_train))

    lr_pred=best_lr_model.predict(X_test)
    # Calcola le metriche per la classificazione
    accuracy_lr = accuracy_score(y_test, lr_pred)

    # Aggiornamento dell'accuratezza totale
    acc_lr += accuracy_lr
      
  #Calcolo della media per sessione
  media_lr = acc_lr / 15

  #Calcolo della media tra le 3 sessioni
  accuracyfinallr +=media_lr

# Stampa della media dell'accuratezza
print("Media dell'accuratezza Logistic Regression:", accuracyfinallr/3)

Media dell'accuratezza Logistic Regression: 0.6277777777777778


## **Naive Bayes**

In [10]:
# Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_nb= 0.0
media_nb= 0.0
accuracyfinalnb= 0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=normalised_data[normalised_data['Key'].str.contains(f"{i}_")]
    #prendiamo un solo partecipante 
    data_par= data[data['Key'].str.contains(partecipante)]
    data_partecipant = data_par.drop(['Key'], axis=1)
    X_train, y_train, X_test, y_test = divide_train_test(np.array(data_partecipant), np.array(etichette_test))

    # Importa il modello migliore 
    best_nb_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_nb_model.pkl")

    # Addestramento dei con tutti i dati di addestramento
    best_nb_model.fit(X_train,np.ravel(y_train))

    nb_pred=best_nb_model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy_nb = accuracy_score(y_test, nb_pred)

    # Aggiornamento dell'accuratezza totale
    acc_nb += accuracy_nb
      
  # Calcolo della media per sessione
  media_nb = acc_nb / 15

  # Calcolo della media tra le 3 sessioni
  accuracyfinalnb +=media_nb

# Stampa della media dell'accuratezza
print("Media dell'accuratezza Naive Bayes:", accuracyfinalnb/3)

Media dell'accuratezza Naive Bayes: 0.65


## **AdaBoost**

In [11]:
# Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_ab = 0.0
media_ab  =0.0
accuracyfinalab=0.0
iterazioni = 0.0

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=normalised_data[normalised_data['Key'].str.contains(f"{i}_")]
    #prendiamo un solo partecipante 
    data_par= data[data['Key'].str.contains(partecipante)]
    data_partecipant = data_par.drop(['Key'], axis=1)
    X_train, y_train, X_test, y_test = divide_train_test(np.array(data_partecipant), np.array(etichette_test))

    # Importa il modello migliore 
    best_ab_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_adaboost_model.pkl")

    # Addestramento dei con tutti i dati di addestramento
    best_ab_model.fit(X_train,np.ravel(y_train))

    ab_pred=best_nb_model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy_ab = accuracy_score(y_test, nb_pred)
  
    # Aggiornamento dell'accuratezza totale
    acc_ab += accuracy_ab
      
  # Calcolo della media per sessione
  media_ab = acc_ab / 15

  # Calcolo della media tra le 3 sessioni
  accuracyfinalab +=media_ab

# Stampa della media dell'accuratezza
print("Media dell'accuratezza AdaBoost:", accuracyfinalab/3)

Media dell'accuratezza AdaBoost: 0.25


## **Codice completo con tutti e 6 i modelli**

In [ ]:
'''# Inizializzazione delle variabili per calcolare la media dell'accuratezza
acc_xgboost,acc_svm,acc_rf,acc_lr, acc_nb,acc_ab = 0.0,0.0,0.0,0.0,0.0,0.0
media_xgboost,media_svm,media_rf,media_lr,media_nb,media_ab  =0.0,0.0,0.0,0.0,0.0,0.0
accuracyfinalxgboost,accuracyfinalvsm,accuracyfinalrf,accuracyfinallr,accuracyfinalnb,accuracyfinalab =0.0,0.0,0.0,0.0,0.0,0.0

# Lista dei partecipanti
lista_partecipanti = ["ldy", "rx", "qyt", "tyc", "ha", "whh", "mz", "mhw", "zjy", "wll" ,"zjd", "hql", "ly", "wq", "cz"]

#per ogni sessione
for i in range(1, 4):
  # Iterazione per chiave del dizionario
  for partecipante in lista_partecipanti:
    # Separare i dati di addestramento e di test
    data=data_eeg[data_eeg['Key'].str.contains(f"{i}_")]
    #prendiamo un solo partecipante 
    data_par= data[data['Key'].str.contains(partecipante)]
    data_partecipant = data_par.drop(['Key'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(data_partecipant, etichette_test, test_size=0.33, random_state=42)

    # Importa il modello migliore 
    best_xgboost_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_xgboost_model.pkl")
    best_svm_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_svm_model.pkl")
    best_rf_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_rf_model.pkl")
    best_lr_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_lr_model.pkl")
    best_nb_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_nb_model.pkl")
    best_ab_model = joblib.load("/content/drive/Shareddrives/biometria_progetto/materiale_SubjectBiased/Modelli/best_adaboost_model.pkl")

    # Addestramento dei con tutti i dati di addestramento
    best_xgboost_model.fit(X_train, np.ravel(y_train))
    best_svm_model.fit(X_train,np.ravel(y_train))
    best_rf_model.fit(X_train, np.ravel(y_train))
    best_lr_model.fit(X_train,np.ravel(y_train))
    best_nb_model.fit(X_train,np.ravel(y_train))
    best_ab_model.fit(X_train,np.ravel(y_train))

    xgboost_pred = best_xgboost_model.predict(X_test)
    svm_pred= best_svm_model.predict(X_test)
    rf_pred=best_rf_model.predict(X_test)
    lr_pred=best_lr_model.predict(X_test)
    nb_pred=best_nb_model.predict(X_test)
    ab_pred=best_nb_model.predict(X_test)

    # Calcola le metriche per la classificazione
    accuracy_xgboost = accuracy_score(y_test, xgboost_pred)
    accuracy_svm = accuracy_score(y_test, svm_pred)
    accuracy_rf = accuracy_score(y_test, rf_pred)
    accuracy_lr = accuracy_score(y_test, lr_pred)
    accuracy_nb = accuracy_score(y_test, nb_pred)
    accuracy_ab = accuracy_score(y_test, nb_pred)
  
    # Aggiornamento dell'accuratezza totale
    acc_xgboost += accuracy_xgboost
    acc_svm += accuracy_svm
    acc_rf += accuracy_rf
    acc_lr += accuracy_lr
    acc_nb += accuracy_nb
    acc_ab += accuracy_ab
      
  # Calcolo della media per sessione
  media_xgboost = acc_xgboost / 15
  media_svm = acc_svm / 15
  media_rf = acc_rf / 15
  media_lr = acc_lr / 15
  media_nb = acc_nb / 15
  media_ab = acc_ab / 15

  # Calcolo della media tra le 3 sessioni
  accuracyfinalxgboost += media_xgboost
  accuracyfinalvsm += media_svm
  accuracyfinalrf +=media_rf
  accuracyfinallr +=media_lr
  accuracyfinalnb +=media_nb
  accuracyfinalab +=media_ab

# Stampa della media dell'accuratezza
print("Media dell'accuratezza XGBOOST:", accuracyfinalxgboost/3)
print("Media dell'accuratezza SMV:", accuracyfinalvsm/3)
print("Media dell'accuratezza RF:", accuracyfinalrf/3)
print("Media dell'accuratezza LR:", accuracyfinallr/3)
print("Media dell'accuratezza NB:", accuracyfinalnb/3)
print("Media dell'accuratezza AB:", accuracyfinalab/3)'''

# **Requirements**

In [12]:
#Comando necessario per ottenere le librerie installate insieme alle loro versioni specifiche.
import subprocess
path = '/content/drive/Shareddrives/biometria_progetto/requirements_SD.txt'
subprocess.run(f'pip freeze > {path}', shell=True, check=True)

CompletedProcess(args='pip freeze > /content/drive/Shareddrives/biometria_progetto/requirements_SD.txt', returncode=0)